## <font color='blue'>This Jupyter Notebook fits corpus to kmeans, plots elbow graphs, plots TSNE, plots kmeans, and prints word clusters</font>

### 1.1 Import necessary packages and define functions

In [1]:
import collections
from cleaner_funcs import clean, clean_text_string, clean_list
import csv
from helpers import top_features, add_stopwords, get_kmeans_wcss, get_top_keywords, plot_tsne_pca, load_csv
import json
import matplotlib.cm as cm
from matplotlib import pyplot as plt
from NLPPipe import NLPPipe
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer, TreebankWordTokenizer
from nltk.util import ngrams
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from textblob import TextBlob

%matplotlib inline

### 1.2 Print Current Working Directory 

In [2]:
os.getcwd()

'C:\\Users\\Meehir\\Documents\\GitHub\\project-4-public\\notebooks'

### 1.3 Read in JSON

In [3]:
data_path = 'C:\\Users\\Meehir\\Documents\\GitHub\\project-4-public\\data\\'

with open(data_path + 'json_files\\combined_2000.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

### 1.4 Build corpus from comment text strings

In [5]:
comms = []

for thread in data:
    for comment in thread['comments']:
        comms.append(comment['comment'])
        
corpus = comms

### 1.5 Loading in stop words. See [preprocess](preprocess.ipynb) notebook for further details

In [6]:
stop_word_loc = data_path + "\\csv_files\\"
my_stop_words = load_csv(data_path)

In [7]:
words_to_del = ['ve', 'thi', 'wa', 'like', 'year','just', 'know', 'good', 'don', 'ani', 'want', 'realli', 'veri',
                'ha', 'thank','say', 'got', 'becaus', 'look', 'make', 'time', 'ha', 'did', 'peopl', 'doe', 'sure',
                'need', 'whi', 'lot', 'thing', 'state', 'gener', 'great', 'anyon', 'probabl', 'hi', 'tri', 'sinc', 
                'ec','way', 'said', 'll', 'write', 'high', 'place', 'didn', 'everyon', 'pretti', 'come', 'ye', 'ca', 'ask', 'els', 'use', 'ii', 'doesn',
                'ivi', 'person', 'http', 'kid', 'day', 'talk', 'number', 'start', 'anoth', 'anoth', 'end', 'rd', 
                'someth', 'mayb', 'anyth', 'possibl', 'thread', 'bc', 'wonder', 'alreadi','took', 'hour', 'www', 
                'everi', 'abl', 'went', 'let', 'load', 'heard', 'tell', 'live', 'sea', 'big', 'pleas',
                'someon', 'chang', 'inform', 'bit', 'urm', 'guess', 'mention', 'taken', 'dure', 'believ', 
                'com', 'especi','isn', 'abov', 'lol', 'compar', 'wrote', 'agre', 'object', 'non', 'noth', 
                'alway', 'edu', 'hey', 'sent','ab', 'kind', 'given', 'overal', 'happen', 'singl', 'pre', 'penn', 'upenn']

In [8]:
my_stop_words = add_stopwords(my_stop_words, words_to_del)

In [9]:
nlp = NLPPipe(vectorizer=TfidfVectorizer(stop_words=set(my_stop_words),max_features=25000), 
              cleaning_function=clean, 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer())

nlp.fit(corpus)
X = nlp.transform(corpus).toarray()
df = pd.DataFrame(X, columns = nlp.vectorizer.get_feature_names())
df

,aa,aaa,aaaa,aaaaa,aaaaahhhhh,aaaand,aaand,aaja,aapplic,aapt,...,zonumh,zoo,zoolog,zoom,zuckerberg,zuckerman,zurich,zxcvbnmwait,zzeemmnntt,état
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
len(nlp.vectorizer.get_feature_names())

In [ ]:
nlp.fit(corpus)
vectors = nlp.transform(corpus)

X_data = vectors.todense()
print(X_data.shape)

wcss=[]

for i in range(1,25): 
    kmeans = KMeans(n_clusters=i, init ='k-means++', max_iter=300,  n_init=10,random_state=0)
    kmeans.fit(X_data)

    wcss.append(kmeans.inertia_)

(16451, 13978)


In [ ]:
len(wcss)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method Graph')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
nlp2, wcss2 = get_kmeans_wcss(my_stop_words, 250, corpus)
nlp3, wcss3 = get_kmeans_wcss(my_stop_words, 2500, corpus)
nlp4, wcss4 = get_kmeans_wcss(my_stop_words, 1000, corpus)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(range(1,11),wcss2, '-*', label = 'top 250 words', linewidth = 4, markersize=25)
plt.plot(range(1, 11), wcss, '-p', label = 'all 13899 words', linewidth = 4,markersize=25)
plt.plot(range(1,11), wcss3, '-h', label = 'top 2500 words', linewidth = 4,markersize=25)
plt.plot(range(1,11), wcss4, '-o', label = 'top 1000 words', linewidth = 4,markersize=25)
plt.title('The Elbow Graph', fontsize=32)
plt.xlabel('Number of clusters', fontsize=20)
plt.ylabel('WCSS', fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize='xx-large')
plt.savefig('elbow.png')

In [ ]:
cols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
ind = ['All 13899 words', 'Top 2500 words', 'Top 1000 words', 'Top 250 words']

In [ ]:
df = pd.DataFrame([wcss, wcss3, wcss4, wcss2], columns = cols)
df = df.set_axis(ind, inplace=False).set_axis(cols, axis=1, inplace=False)
df

In [ ]:
kmeans = KMeans(n_clusters=7, init ='k-means++', max_iter=300, n_init=10,random_state=0)
clusters = kmeans.fit_predict(X_data)

In [ ]:
def get_top_keywords(df, clusters, labels, n_terms):
    
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]))

In [ ]:
get_top_keywords(df, clusters, nlp.vectorizer.get_feature_names(), 10)

In [ ]:
plot_tsne_pca(X_data, clusters)